# Ollama + OpenAI + Python

## 1. Chỉ định tên mô hình

Nếu bạn đã tải một mô hình khác ngoài "phi3:mini", hãy thay đổi giá trị trong ô dưới đây. Biến đó sẽ được sử dụng trong mã xuyên suốt notebook.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Cài đặt Open AI client

Thông thường, OpenAI client được sử dụng với OpenAI.com hoặc Azure OpenAI để tương tác với các mô hình ngôn ngữ lớn.  
Tuy nhiên, nó cũng có thể được sử dụng với Ollama, vì Ollama cung cấp một endpoint tương thích với OpenAI tại "http://localhost:11434/v1".


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Tạo một cuộc trò chuyện hoàn chỉnh

Bây giờ chúng ta có thể sử dụng OpenAI SDK để tạo phản hồi cho một cuộc trò chuyện. Yêu cầu này sẽ tạo ra một bài haiku về mèo:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Kỹ thuật tạo lời nhắc

Tin nhắn đầu tiên gửi đến mô hình ngôn ngữ được gọi là "tin nhắn hệ thống" hoặc "lời nhắc hệ thống", và nó thiết lập hướng dẫn tổng thể cho mô hình.  
Bạn có thể cung cấp lời nhắc hệ thống tùy chỉnh của riêng mình để hướng dẫn mô hình ngôn ngữ tạo ra đầu ra theo cách khác biệt.  
Hãy chỉnh sửa `SYSTEM_MESSAGE` dưới đây để trả lời giống như nhân vật yêu thích của bạn trong một bộ phim hoặc chương trình truyền hình nổi tiếng, hoặc lấy cảm hứng từ các lời nhắc hệ thống khác tại [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts).

Sau khi bạn đã tùy chỉnh tin nhắn hệ thống, hãy cung cấp câu hỏi đầu tiên của người dùng trong `USER_MESSAGE`.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Một vài ví dụ minh họa

Một cách khác để hướng dẫn mô hình ngôn ngữ là cung cấp "một vài ví dụ", tức là một chuỗi các câu hỏi/đáp án mẫu để minh họa cách nó nên phản hồi.

Ví dụ dưới đây cố gắng khiến mô hình ngôn ngữ hoạt động như một trợ giảng bằng cách cung cấp một vài ví dụ về câu hỏi và câu trả lời mà một trợ giảng có thể đưa ra, sau đó đưa ra câu hỏi mà một sinh viên có thể hỏi.

Hãy thử trước, sau đó chỉnh sửa `SYSTEM_MESSAGE`, `EXAMPLES`, và `USER_MESSAGE` cho một tình huống mới.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Tạo nội dung tăng cường truy xuất

RAG (Tạo nội dung tăng cường truy xuất) là một kỹ thuật giúp mô hình ngôn ngữ trả lời câu hỏi chính xác trong một lĩnh vực cụ thể, bằng cách trước tiên truy xuất thông tin liên quan từ một nguồn kiến thức và sau đó tạo ra câu trả lời dựa trên thông tin đó.

Chúng tôi đã cung cấp một tệp CSV cục bộ chứa dữ liệu về xe hơi lai. Đoạn mã dưới đây đọc tệp CSV, tìm kiếm các kết quả phù hợp với câu hỏi của người dùng, và sau đó tạo ra câu trả lời dựa trên thông tin tìm được. Lưu ý rằng điều này sẽ mất nhiều thời gian hơn so với các ví dụ trước, vì nó gửi nhiều dữ liệu hơn đến mô hình. Nếu bạn nhận thấy câu trả lời vẫn chưa dựa trên dữ liệu, bạn có thể thử điều chỉnh hệ thống hoặc thử các mô hình khác. Nhìn chung, RAG hiệu quả hơn với các mô hình lớn hoặc các phiên bản SLM được tinh chỉnh.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp từ con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
